In [48]:
import os
import anndata
import pandas as pd
import anndata as ad
from FlowCytometryTools import FCMeasurement
import re
import numpy as np

### Create functions to load and concatenate data

In [49]:
def arcsinh_transform(X,cofactor=5):
    return np.arcsinh(X/cofactor)

In [51]:
def load_data_fcs(path,condition_name):
    data = FCMeasurement(ID="Sample", datafile=path)
    anndata = ad.AnnData(data.data)
    
    pattern = r"_(\w+)\s*-\s*(\w+)\.fcs$"
    match_stim = re.search(pattern, path)
    if match_stim:
        #stim_type = match_stim.group(1)
        cell_type = match_stim.group(2) 
        anndata.obs['drug']= condition_name
        anndata.obs['cell_type']= cell_type
    else:
        print('No cell_type found, bad format')
    anndata.X=arcsinh_transform(anndata.X)
    return anndata

In [52]:
def concatenate_2conditions_data(path_stim,stim_name,path_unstim,unstim_name,outdir_path):
    unstim_anndata = load_data_fcs(path_unstim,unstim_name)
    stim_anndata = load_data_fcs(path_stim,stim_name)

    combined_anndata = stim_anndata.concatenate(unstim_anndata, batch_key="condition", batch_categories=["stim", "control"])
    combined_anndata.write(outdir_path)
    return

In [41]:
def concatenate_2conditions_multiple_data(path_stim_list,stim_name,path_unstim_list,unstim_name,outdir_path):
    comb_unstim_anndata = load_data_fcs(path_unstim_list[0],unstim_name)
    comb_stim_anndata = load_data_fcs(path_stim_list[0],stim_name)
    compt_un=1
    compt_st=1
    for path_stim,path_unstim in zip(path_stim_list[1:],path_unstim_list[1:]):
        unstim_anndata = load_data_fcs(path_unstim,unstim_name)
        stim_anndata = load_data_fcs(path_stim,stim_name)

        comb_unstim_anndata = comb_unstim_anndata.concatenate(unstim_anndata)
        comb_stim_anndata = comb_stim_anndata.concatenate(stim_anndata)
        compt_un+=1
        compt_st+=1
        print(compt_un,compt_st)
    combined_anndata=comb_stim_anndata.concatenate(comb_unstim_anndata, batch_key="condition", batch_categories=["stim", "control"])
    
    combined_anndata.write(outdir_path)
    return 

In [42]:
def concatenate_1condition_multiple_data(path_list,condition_name,outdir_path):
    comb_anndata = load_data_fcs(path_list[0],condition_name)

    for path in path_list[1:]:
        new_anndata = load_data_fcs(path,condition_name)
        comb_anndata = comb_anndata.concatenate(new_anndata)
    comb_anndata.write(outdir_path)
    return 

In [67]:
def create_list_of_paths(directory,stimulation,cell_type=None,sample=None,patient_excluded=None):
    paths_list = []
    for filename in os.listdir(directory):
        if stimulation in filename and (cell_type is None or cell_type in filename) and (sample is None or sample in filename) and (patient_excluded is None or patient_excluded not in filename):
            paths_list.append(os.path.join(directory, filename))
    return paths_list


###  Examples

In [115]:
path_unstim='datasets/atest_data/controls_1_Surge_3_HCAA_BL_Unstim - Bcells.fcs'
path_stim = "datasets/atest_data/controls_1_Surge_3_IC1_IC1_Stim_1 - Bcells.fcs"
outdir_path = "datasets/atest_data/combined_Bcell_norm.h5ad"
concatenate_2conditions_data(path_stim,'Stim_1_Bcell', path_unstim,'Unstim_Bcell', outdir_path)
combined_anndata=ad.read("datasets/atest_data/combined_Bcell_norm.h5ad")

/Users/MacBook/miniconda3/envs/cellot_env/lib/python3.8/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
... storing 'drug' as categorical
... storing 'cell_type' as categorical


In [54]:
directory = '/Users/MacBook/stanford/cellot/datasets/surge_prehab_controls - Gated Populations'
stimulations = ['LPS','Unstim']
def count_files(directory,stimulation):
    count = 0
    sumsize = 0
    for filename in os.listdir(directory):
        if filename.endswith(".fcs") and stimulation in filename:
            count+=1
            sumsize+=os.path.getsize(os.path.join(directory, filename))
    return count,sumsize
for stimulation in stimulations:
    print(stimulation,count_files(directory,stimulation))

LPS (126, 849626243)
Unstim (133, 1048400869)


In [68]:
path_unstim_list=create_list_of_paths(directory='/Users/MacBook/stanford/cellot/datasets/surge_prehab_controls - Gated Populations',stimulation='Unstim',cell_type='Bcells',patient_excluded='HCAA')
path_stim_list=create_list_of_paths(directory='/Users/MacBook/stanford/cellot/datasets/surge_prehab_controls - Gated Populations',stimulation='LPS',cell_type='Bcells',patient_excluded='HCAA')
res=concatenate_2conditions_multiple_data(path_stim_list,'LPS',path_unstim_list,'Unstim',"datasets/sherlock_training_data/combined_LPS.h5ad")


/Users/MacBook/miniconda3/envs/cellot_env/lib/python3.8/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16


... storing 'drug' as categorical
... storing 'cell_type' as categorical


In [61]:
path_unstim_list

['/Users/MacBook/stanford/cellot/datasets/surge_prehab_controls - Gated Populations/controls_1_Surge_3_HCBB_BL_Unstim - Bcells.fcs',
 '/Users/MacBook/stanford/cellot/datasets/surge_prehab_controls - Gated Populations/controls_1_Surge_3_HCBB_IDX_Unstim - Bcells.fcs']

In [14]:
len(path_stim_list),len(path_unstim_list)   

(18, 19)

### Pipeline for evaluation

In [ ]:
eval_4i=pd.read_csv("/Users/MacBook/stanford/cellot/results/4i/drug-cisplatin/model-cellot/evals_iid_data_space/evals.csv")

In [10]:
print('eval metric available:',eval_4i['metric'].unique())

eval metric available: ['l2-means' 'l2-stds' 'r2-means' 'r2-stds' 'r2-pairwise_feat_corrs'
 'l2-pairwise_feat_corrs' 'mmd' 'enrichment-k50' 'enrichment-k100']


In [12]:
eval_4i.ncells.unique()

array([100, 250, 500])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
def create_density_plots(dist_data, out_file, title_suffix=""):
    """
    Create density plots for Unstim, Stim True, and Stim Pred distributions.
    """
    sns.set_theme(style="whitegrid")
    pts_sorted = sorted(dist_data.keys())

    fig = plt.figure(figsize=(18, 18))
    fig.suptitle(f"Density Plots {title_suffix}", fontsize=16)

    rows = int(np.ceil(len(pts_sorted) / 3))
    for i, pt in enumerate(pts_sorted, start=1):
        ax = fig.add_subplot(rows, 3, i)
        cat_labels = ["Unstim", "Stim True", "Stim Pred"]
        cat_colors = ["blue", "red", "green"]
        for label, color in zip(cat_labels, cat_colors):
            arr = dist_data[pt][label]
            if arr.size > 0:
                sns.kdeplot(arr, ax=ax, label=f"{label} (n={arr.size})", color=color, fill=False)
        ax.set_title(f"Patient: {pt}")
        ax.legend(fontsize=8)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(out_file, dpi=150)
    plt.close()

In [ ]:
target=ad.read('./datasets/atest_data/combined_Bcell.h5ad')
target1=target[:,'150Nd_pSTAT5'].copy()
t=pd.DataFrame(target1.X)
stim=pd.DataFrame(target1[target1.obs['condition']=='stim'].X)
unstim=pd.DataFrame(target1[target1.obs['condition']=='control'].X)

In [96]:
def plot_result(prediction_path,original_path,marker,outdir_path):
    target=ad.read(original_path)
    target1=target[:,marker].copy()
    stim=pd.DataFrame(target1[target1.obs['condition']=='stim'].X)
    unstim=pd.DataFrame(target1[target1.obs['condition']=='control'].X)

    dataf=pd.read_csv(prediction_path)
    dataf["Stim Pred"]=dataf[marker]
    dataf['Stim True']=stim.iloc[:,0]
    dataf['Unstim']=unstim.iloc[:,0]
    
    dist_data = {
    "Patient_1": {
        "Stim True": dataf["Stim True"].values,
        "Stim Pred": dataf["Stim Pred"].values,
        "Unstim": dataf["Unstim"].values}}

    create_density_plots(dist_data, outdir_path, title_suffix="")     
    return dataf[['Stim True','Stim Pred','Unstim']]

In [ ]:
EVALUATION_MARKERS = [
    '151Eu_pp38', '155Gd_pS6', '149Sm_pCREB', 
    '159Tb_pMAPKAPK2', '166Er_pNFkB', '167Er_pERK12'
]

In [98]:
plot_result('/Users/MacBook/stanford/cellot/results/test_new/unseen_data_result/prediction.csv','./datasets/atest_data/combined_Bcell.h5ad','150Nd_pSTAT5',"/Users/MacBook/stanford/cellot/plots/density_plot_test1.png")

,Stim True,Stim Pred,Unstim
0,0.000000,7.837983,0.00000
1,9.945632,3.335905,0.00000
2,6.893324,14.173313,0.00000
3,21.470419,3.335905,1.05323
4,16.403343,3.335905,0.00000
...,...,...,...
5934,5.409459,14.412180,0.00000
5935,6.349995,8.230350,0.00000
5936,3.295533,3.335905,0.00000
5937,2.568134,3.335905,0.00000
